In [22]:
from random import seed
from random import randrange
from math import floor
from math import exp
import pandas as pd #Importar dataset

In [23]:
#Clase para dividir el dataset en Entrenamiento y Testeo
def cross_val_split(data_X,data_Y,test_size,seed_val):
    data_x = data_X.tolist()
    data_y = data_Y.tolist()
    seed(seed_val)
    train_size = floor((1 - test_size)*len(data_x))
    train_x = []
    train_y = []
    while(len(train_x)<train_size):
        index = randrange(len(data_x))
        train_x.append(data_x.pop(index))
        train_y.append(data_y.pop(index))
    return train_x,train_y,data_x,data_y

In [24]:
#Función para devolver las estadísticas de minimos y máximos de la columna para el escalado
def statistics(x):
    cols = list(zip(*x))
    stats = []
    for e in cols:
        stats.append([min(e),max(e)])
    return stats

In [25]:
#Función para escalar las características
def scale(x, stat):
    for row in x:
        for i in range(len(row)):
            row[i] = (row[i] - stat[i][0])/(stat[i][1] - stat[i][0])

In [26]:
#Función para convertir diferentes clases en diferentes columnas(Separar clases) 
def one_vs_all_cols(s):
    m = list(set(s))
    m.sort() #Ordenamos los datos
    for i in range(len(s)):
        new = [0]*len(m)
        new[m.index(s[i])] = 1
        s[i] = new
    return m

In [7]:
#Función para calcular "Theta transpose"
def ThetaTX(Q,X):
    det = 0.0
    for i in range(len(Q)):
        det += X[i]*Q[i]
    return det

In [27]:
#Función para calcular el costo por clase negativa (classs = 0)
def LinearSVM_cost0(z):
    if(z < -1): #Margen
        return 0
    return z + 1

In [28]:
#Función para calcular el costo por clase positiva (classs = 1)
def LinearSVM_cost1(z):
    if(z > 1): #Margen
        return 0
    return -z + 1

In [29]:
#Función pra calcular la Sigmoide
def sigmoid(z):
    return 1.0/(1.0 + exp(-z))

In [30]:
#Función para calcular el costo de  SVM
def cost(theta,c,x,y):
    cost = 0.0
    for i in range(len(x)):
        z = ThetaTX(theta[c], x[i])
        cost += y[i]*LinearSVM_cost1(z) + (1 - y[i])*LinearSVM_cost0(z)
    return cost

In [31]:
#Función para realizar el descenso del gradiente en los pesos / parámetros 
#(Esto es escencial cuando se usa la función sigmoide para entrenar un SVM,  ya que permite 
#encontrar los valores mínimos de esta función)
def gradDescent(theta,c,x,y,learning_rate):
    oldTheta = theta[c]
    for Q in range(len(theta[c])):
        derivative_sum = 0 
        for i in range(len(x)):
            derivative_sum += (sigmoid(ThetaTX(oldTheta,x[i])) - y[i])*x[i][Q]
        theta[c][Q] -= learning_rate*derivative_sum

In [32]:
#Función para devolver las predicciones
def predict(data,theta):
    predictions = []
    count = 1
    for row in data:
        hypothesis = []
        multiclass_ans = [0]*len(theta)
        for c in range(len(theta)):
            z = ThetaTX(row,theta[c])
            hypothesis.append(sigmoid(z))
        index = hypothesis.index(max(hypothesis))
        multiclass_ans[index] = 1
        predictions.append(multiclass_ans)
        count+=1
    return predictions

In [33]:
#Función para devolver la predicción y calcular el porcentaje de predicción
def accuracy(predicted, actual):
    n = len(predicted)
    correct = 0
    for i in range(n):
        if(predicted[i]==actual[i]):
            correct+=1
    return correct/n

In [34]:
#Función para realizar validación cruzada
#Aqui calculamos la media arirmética de cada clase para poder evaluar y garantizar que los datos 
#obtenidos son independientes de los datos de entrenamiento y prueba
def cross_validation(x,y,test_data_size,validations,learning_rate,epoch):
    accuracies = []
    for valid in range(validations):
        x_train, y_train, x_test, y_test = cross_val_split(x,y,test_data_size,valid+1)
        #Convertir y_train a columnas con valores 0/1
        classes = []
        for i in range(len(label_map)):
            classes.append([row[i] for row in y_train])
        #Inicializando función Theta (Pesos)
        theta = [[0]*len(x_train[0]) for _ in range(len(classes))]
        
        #Modelo de entrenamiento
        for i in range(epoch):
            for class_type in range(len(classes)):
                gradDescent(theta,class_type,x_train,classes[class_type],learning_rate)
            if(i%(epoch/10)==0):
                print("Procesando", i*100/epoch,"%")
        print("Completado")
        
        #Predicción usando datos de prueba 
        y_pred = predict(x_test,theta)
        
        #Cálculo de al presición
        accuracies.append(accuracy(y_pred,y_test))
    return sum(accuracies)/len(accuracies)

In [35]:
#Importar Dataset
print("SVM Lineal- Función Sigmoide")
dataset = pd.read_csv("dataset1.csv")#Importamos dataset
data = dataset.values

#Separamos a las clases de la etiqueta
x = data[:,:(data.shape[1]-2)]
y = data[:,(data.shape[1]-1)]
#Aqui separamos las carácticas más notorias en mínimos y máximos 
stats = statistics(x)
scale(x,stats)

#Convertimos las etiquetas a columnas
#Usamos esta función para recuperar la etiqueta original (String)
label_map = one_vs_all_cols(y)

#Separamos el dataset de Datos de entrenamiento y datos de prueba
test_data_size = 0.2
learning_rate = 0.01

epoch = 20 #Tiempo (ms) para evaluar (Mientas más tiempo- aumenta el porcentaje de aciertos)
validations = 1 #Número de validaciones 
final_score = cross_validation(x,y,test_data_size,validations,learning_rate,epoch)
#Resultados
print("Datos: ", (floor(len(x)*(1 - test_data_size)))+(len(x) - floor(len(x)*(1 - test_data_size))))
print("Tamaño datos de Entrenamiento: ", floor(len(x)*(1 - test_data_size)))
print("Tamaño datos de Prueba: ", len(x) - floor(len(x)*(1 - test_data_size)))
print("Porcentaje de aciertos: ",final_score*100,"%")

SVM Lineal- Función Sigmoide
Procesando 0.0 %
Procesando 10.0 %
Procesando 20.0 %
Procesando 30.0 %
Procesando 40.0 %
Procesando 50.0 %
Procesando 60.0 %
Procesando 70.0 %
Procesando 80.0 %
Procesando 90.0 %
Completado
Datos:  522
Tamaño datos de Entrenamiento:  417
Tamaño datos de Prueba:  105
Porcentaje de aciertos:  84.76190476190476 %
